In [29]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

In [25]:
model_id = "GEB-AGI/geb-1.3b"
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).bfloat16() #.cuda()

tokenizer = AutoTokenizer.from_pretrained("/home/javen/Projects/geb-1.3b", trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [8]:
"""
Load WikiHow dataset.

https://huggingface.co/datasets/gursi26/wikihow-cleaned
https://github.com/mahnazkoupaee/WikiHow-Dataset
"""
dataset = load_dataset("gursi26/wikihow-cleaned")
dataset = dataset['train']

In [30]:
"""
Tokenize data.
"""
tokenized_text = tokenizer(dataset["text"][0], return_tensors="np", padding=True)

# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_text = dict(tokenized_text)

summaries = np.array(dataset["summary"])